In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


part a) Accuracy of relu is better than sigmoid in both training and testing<br>
As there is non saturation of gradients in relu and in sigmoid they get saturated, relu is sparse and has induced regularization. Relu runs faster than sigmoid as it is simpler to compute. 

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [4]:
use_cuda = torch.cuda.is_available()    
torch.manual_seed(1)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True, **kwargs)

In [5]:
epochs = 5

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=00.01, momentum=0.5)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    print("Train set")
    test(model, device, train_loader)
    print("Test set")
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300039
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.213470
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.170460
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.076699
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.868078
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.414199
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.000870
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.775734
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.460020
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.486316
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.437761
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.410437
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.462436
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.429283
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.398867
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.386359
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.298190
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.504132
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.522952
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.337516
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.027017
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.021184
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.025400
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.095996
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.137479
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.211134
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.011439
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.042422
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.058756
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.071311
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.057006
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.037236
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.042545
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.023623
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.044068
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.058392
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.133838
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.058626
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.018346
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.149680


Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.033535
Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.038986
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.011108
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.016350
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.097144
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.019332
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.011676
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.042241
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.004987
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.143680
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.025358
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.126773
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.024116
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.021720
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.078310
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.037237
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.062419
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.044536
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.049137
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.037225


4a) sigmoid

In [7]:
class SigmoidNet(nn.Module):
    def __init__(self):
        super(SigmoidNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.sigmoid(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.sigmoid(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = SigmoidNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=00.01, momentum=0.5)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    print("Train set")
    test(model, device, train_loader)
    print("Test set")
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.292659


/home/devansh/anaconda3/envs/pytorchEnv/lib/python3.7/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [640/60000 (1%)]	Loss: 2.328458
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.339575
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.322630
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.301530
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.303971
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.322420
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.320086
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.303635
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.296306
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.293317
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.324865
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.295942
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.314339
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.316056
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.301824
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.288373
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.306030
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.309470
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.306659
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.282547
Train Epoch: 1 [13440/60

Train Epoch: 2 [44800/60000 (75%)]	Loss: 2.293797
Train Epoch: 2 [45440/60000 (76%)]	Loss: 2.302306
Train Epoch: 2 [46080/60000 (77%)]	Loss: 2.263053
Train Epoch: 2 [46720/60000 (78%)]	Loss: 2.281526
Train Epoch: 2 [47360/60000 (79%)]	Loss: 2.291195
Train Epoch: 2 [48000/60000 (80%)]	Loss: 2.281653
Train Epoch: 2 [48640/60000 (81%)]	Loss: 2.275370
Train Epoch: 2 [49280/60000 (82%)]	Loss: 2.281388
Train Epoch: 2 [49920/60000 (83%)]	Loss: 2.316043
Train Epoch: 2 [50560/60000 (84%)]	Loss: 2.277465
Train Epoch: 2 [51200/60000 (85%)]	Loss: 2.279663
Train Epoch: 2 [51840/60000 (86%)]	Loss: 2.271987
Train Epoch: 2 [52480/60000 (87%)]	Loss: 2.276724
Train Epoch: 2 [53120/60000 (88%)]	Loss: 2.282404
Train Epoch: 2 [53760/60000 (90%)]	Loss: 2.263318
Train Epoch: 2 [54400/60000 (91%)]	Loss: 2.261201
Train Epoch: 2 [55040/60000 (92%)]	Loss: 2.268203
Train Epoch: 2 [55680/60000 (93%)]	Loss: 2.264881
Train Epoch: 2 [56320/60000 (94%)]	Loss: 2.291643
Train Epoch: 2 [56960/60000 (95%)]	Loss: 2.262630


Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.960687
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.969875
Train Epoch: 4 [28800/60000 (48%)]	Loss: 1.032002
Train Epoch: 4 [29440/60000 (49%)]	Loss: 1.037740
Train Epoch: 4 [30080/60000 (50%)]	Loss: 1.142515
Train Epoch: 4 [30720/60000 (51%)]	Loss: 1.005519
Train Epoch: 4 [31360/60000 (52%)]	Loss: 1.102189
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.990388
Train Epoch: 4 [32640/60000 (54%)]	Loss: 1.027275
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.854354
Train Epoch: 4 [33920/60000 (57%)]	Loss: 1.082792
Train Epoch: 4 [34560/60000 (58%)]	Loss: 1.017896
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.917076
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.995445
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.810790
Train Epoch: 4 [37120/60000 (62%)]	Loss: 1.073644
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.955762
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.954344
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.807228
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.887345


part b)<br>
0.25:<br>
Train set<br>

Average loss: 0.0426, Accuracy: 59453/60000 (99%)<br>

Test set<br>

Average loss: 0.0416, Accuracy: 9914/10000 (99%)<br>

0.5<br>
Average loss: 0.1161, Accuracy: 59428/60000 (99%)<br>

Test set<br>

Average loss: 0.1120, Accuracy: 9895/10000 (99%)<br>

0.75<br>
Train set<br>

Average loss: 0.4457, Accuracy: 59102/60000 (99%)<br>

Test set<br>

Average loss: 0.4329, Accuracy: 9869/10000 (99%)<br>

1<br>
Train set<br>

Average loss: 2.3013, Accuracy: 6661/60000 (11%)<br>

Test set<br>

Average loss: 2.3018, Accuracy: 1128/10000 (11%)<br>

Error is high at probability = 1 this is expected as all the weights get dropped. Dropout increases variances of the model and if we increase the dropout too much the variances increases a lot, the model becomes less complex and is not able to learn properly. This all happens after a thresold. The accuracy of the model decreases as dropout increases. Best dropout setting is 0.25

In [10]:
class DropoutReluNet(nn.Module):
    def __init__(self, P):
        super(DropoutReluNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        self.dropout = nn.Dropout(p= P)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.dropout(x)
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = self.dropout(x)
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [11]:
for P in [.25, .5, .75, 1]:
    print("dropout probability = ", P) 
    model = DropoutReluNet(P).to(device)
    optimizer = optim.SGD(model.parameters(), lr=00.01, momentum=0.5)

    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        print("Train set")
        test(model, device, train_loader)
        print("Test set")
        test(model, device, test_loader)

dropout probability =  0.25
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.317758
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.213991
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.019415
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.643039
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.318615
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.970965
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.601724
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.692490
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.681940
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.588948
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.427867
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.385439
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.451455
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.426072
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.359919
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.355311
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.417321
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.269041
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.506293
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.268205
T

Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.065739
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.092974
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.213045
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.015236
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.106271
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.132812
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.132802
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.066028
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.016069
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.073973
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.129115
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.042315
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.184824
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.025361
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.171605
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.122664
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.053480
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.100005
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.013318
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.063425


Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.043347
Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.041271
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.029665
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.016363
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.103073
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.039735
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.057894
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.056346
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.022786
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.070883
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.032911
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.020033
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.076090
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.058100
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.059432
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.055491
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.044135
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.104058
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.024344
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.028225


Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.252955
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.450171
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.412192
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.210666
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.296918
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.433596
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.354029
Train Epoch: 1 [13440/60000 (22%)]	Loss: 0.229242
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.259445
Train Epoch: 1 [14720/60000 (25%)]	Loss: 0.425348
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.199177
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.313339
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.390456
Train Epoch: 1 [17280/60000 (29%)]	Loss: 0.203750
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.181697
Train Epoch: 1 [18560/60000 (31%)]	Loss: 0.236704
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.346438
Train Epoch: 1 [19840/60000 (33%)]	Loss: 0.301980
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.223463
Train Epoch: 1 [21120/60000 (35%)]	Loss: 0.269504
Tr

Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.186660
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.049848
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.144356
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.157421
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.039925
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.131454
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.185062
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.038099
Train Epoch: 2 [58240/60000 (97%)]	Loss: 0.148414
Train Epoch: 2 [58880/60000 (98%)]	Loss: 0.039981
Train Epoch: 2 [59520/60000 (99%)]	Loss: 0.139180
Train set

Average loss: 0.1958, Accuracy: 58978/60000 (98%)

Test set

Average loss: 0.1851, Accuracy: 9837/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.032346
Train Epoch: 3 [640/60000 (1%)]	Loss: 0.078109
Train Epoch: 3 [1280/60000 (2%)]	Loss: 0.039037
Train Epoch: 3 [1920/60000 (3%)]	Loss: 0.187420
Train Epoch: 3 [2560/60000 (4%)]	Loss: 0.087764
Train Epoch: 3 [3200/60000 (5%)]	Loss: 0.030546
Train Epoch: 3 [3840/60000 (6%)]	Loss: 0.170

...

Train Epoch: 3 [640/60000 (1%)]	Loss: 0.247486
Train Epoch: 3 [1280/60000 (2%)]	Loss: 0.175285
Train Epoch: 3 [1920/60000 (3%)]	Loss: 0.156759
Train Epoch: 3 [2560/60000 (4%)]	Loss: 0.262885
Train Epoch: 3 [3200/60000 (5%)]	Loss: 0.300884
Train Epoch: 3 [3840/60000 (6%)]	Loss: 0.092257
Train Epoch: 3 [4480/60000 (7%)]	Loss: 0.305928
Train Epoch: 3 [5120/60000 (9%)]	Loss: 0.191693
Train Epoch: 3 [5760/60000 (10%)]	Loss: 0.192831
Train Epoch: 3 [6400/60000 (11%)]	Loss: 0.128838
Train Epoch: 3 [7040/60000 (12%)]	Loss: 0.205638
Train Epoch: 3 [7680/60000 (13%)]	Loss: 0.261268
Train Epoch: 3 [8320/60000 (14%)]	Loss: 0.403232
Train Epoch: 3 [8960/60000 (15%)]	Loss: 0.089918
Train Epoch: 3 [9600/60000 (16%)]	Loss: 0.418137
Train Epoch: 3 [10240/60000 (17%)]	Loss: 0.220728
Train Epoch: 3 [10880/60000 (18%)]	Loss: 0.250265
Train Epoch: 3 [11520/60000 (19%)]	Loss: 0.157090
Train Epoch: 3 [12160/60000 (20%)]	Loss: 0.091256
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.082289
Train Epoch: 3 [13440/60

Train Epoch: 4 [44800/60000 (75%)]	Loss: 0.090814
Train Epoch: 4 [45440/60000 (76%)]	Loss: 0.249242
Train Epoch: 4 [46080/60000 (77%)]	Loss: 0.276370
Train Epoch: 4 [46720/60000 (78%)]	Loss: 0.192871
Train Epoch: 4 [47360/60000 (79%)]	Loss: 0.163960
Train Epoch: 4 [48000/60000 (80%)]	Loss: 0.219919
Train Epoch: 4 [48640/60000 (81%)]	Loss: 0.387181
Train Epoch: 4 [49280/60000 (82%)]	Loss: 0.400435
Train Epoch: 4 [49920/60000 (83%)]	Loss: 0.109634
Train Epoch: 4 [50560/60000 (84%)]	Loss: 0.087852
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.072455
Train Epoch: 4 [51840/60000 (86%)]	Loss: 0.139985
Train Epoch: 4 [52480/60000 (87%)]	Loss: 0.085381
Train Epoch: 4 [53120/60000 (88%)]	Loss: 0.213803
Train Epoch: 4 [53760/60000 (90%)]	Loss: 0.186521
Train Epoch: 4 [54400/60000 (91%)]	Loss: 0.166830
Train Epoch: 4 [55040/60000 (92%)]	Loss: 0.164058
Train Epoch: 4 [55680/60000 (93%)]	Loss: 0.222012
Train Epoch: 4 [56320/60000 (94%)]	Loss: 0.072977
Train Epoch: 4 [56960/60000 (95%)]	Loss: 0.321969


Train Epoch: 1 [26880/60000 (45%)]	Loss: 2.296394
Train Epoch: 1 [27520/60000 (46%)]	Loss: 2.295186
Train Epoch: 1 [28160/60000 (47%)]	Loss: 2.302136
Train Epoch: 1 [28800/60000 (48%)]	Loss: 2.310471
Train Epoch: 1 [29440/60000 (49%)]	Loss: 2.289241
Train Epoch: 1 [30080/60000 (50%)]	Loss: 2.307675
Train Epoch: 1 [30720/60000 (51%)]	Loss: 2.303393
Train Epoch: 1 [31360/60000 (52%)]	Loss: 2.309706
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.301742
Train Epoch: 1 [32640/60000 (54%)]	Loss: 2.299521
Train Epoch: 1 [33280/60000 (55%)]	Loss: 2.307532
Train Epoch: 1 [33920/60000 (57%)]	Loss: 2.308872
Train Epoch: 1 [34560/60000 (58%)]	Loss: 2.302116
Train Epoch: 1 [35200/60000 (59%)]	Loss: 2.299877
Train Epoch: 1 [35840/60000 (60%)]	Loss: 2.313793
Train Epoch: 1 [36480/60000 (61%)]	Loss: 2.298132
Train Epoch: 1 [37120/60000 (62%)]	Loss: 2.302469
Train Epoch: 1 [37760/60000 (63%)]	Loss: 2.303616
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.298137
Train Epoch: 1 [39040/60000 (65%)]	Loss: 2.301044


Train Epoch: 3 [9600/60000 (16%)]	Loss: 2.300346
Train Epoch: 3 [10240/60000 (17%)]	Loss: 2.301615
Train Epoch: 3 [10880/60000 (18%)]	Loss: 2.314889
Train Epoch: 3 [11520/60000 (19%)]	Loss: 2.314367
Train Epoch: 3 [12160/60000 (20%)]	Loss: 2.296196
Train Epoch: 3 [12800/60000 (21%)]	Loss: 2.297917
Train Epoch: 3 [13440/60000 (22%)]	Loss: 2.309698
Train Epoch: 3 [14080/60000 (23%)]	Loss: 2.307119
Train Epoch: 3 [14720/60000 (25%)]	Loss: 2.287904
Train Epoch: 3 [15360/60000 (26%)]	Loss: 2.295486
Train Epoch: 3 [16000/60000 (27%)]	Loss: 2.307340
Train Epoch: 3 [16640/60000 (28%)]	Loss: 2.314368
Train Epoch: 3 [17280/60000 (29%)]	Loss: 2.295612
Train Epoch: 3 [17920/60000 (30%)]	Loss: 2.303584
Train Epoch: 3 [18560/60000 (31%)]	Loss: 2.303067
Train Epoch: 3 [19200/60000 (32%)]	Loss: 2.298987
Train Epoch: 3 [19840/60000 (33%)]	Loss: 2.310797
Train Epoch: 3 [20480/60000 (34%)]	Loss: 2.300339
Train Epoch: 3 [21120/60000 (35%)]	Loss: 2.304295
Train Epoch: 3 [21760/60000 (36%)]	Loss: 2.306103
T

Train Epoch: 4 [53760/60000 (90%)]	Loss: 2.308702
Train Epoch: 4 [54400/60000 (91%)]	Loss: 2.303643
Train Epoch: 4 [55040/60000 (92%)]	Loss: 2.301951
Train Epoch: 4 [55680/60000 (93%)]	Loss: 2.298351
Train Epoch: 4 [56320/60000 (94%)]	Loss: 2.297961
Train Epoch: 4 [56960/60000 (95%)]	Loss: 2.295594
Train Epoch: 4 [57600/60000 (96%)]	Loss: 2.304406
Train Epoch: 4 [58240/60000 (97%)]	Loss: 2.307832
Train Epoch: 4 [58880/60000 (98%)]	Loss: 2.296198
Train Epoch: 4 [59520/60000 (99%)]	Loss: 2.303186
Train set

Average loss: 2.3010, Accuracy: 7902/60000 (13%)

Test set

Average loss: 2.3014, Accuracy: 1328/10000 (13%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 2.303406
Train Epoch: 5 [640/60000 (1%)]	Loss: 2.308868
Train Epoch: 5 [1280/60000 (2%)]	Loss: 2.296798
Train Epoch: 5 [1920/60000 (3%)]	Loss: 2.308929
Train Epoch: 5 [2560/60000 (4%)]	Loss: 2.304868
Train Epoch: 5 [3200/60000 (5%)]	Loss: 2.296355
Train Epoch: 5 [3840/60000 (6%)]	Loss: 2.297426
Train Epoch: 5 [4480/60000 (7%)]	Loss: 2.292667

partc)Batch norm + dropout(0.25):<br>
Train set<br>

Average loss: 0.0279, Accuracy: 59598/60000 (99%)<br>

Test set<br>

Average loss: 0.0318, Accuracy: 9915/10000 (99%)<br>

Only Batch norm:<br>
Test set:<br>
Average loss: 0.0146, Accuracy: 59806/60000 (100%)<br>

Test set<br>

Average loss: 0.0287, Accuracy: 9912/10000 (99%)<br>

Both are performing nearly same as both are regularization methods and they each others job, in many papers
they claim that one is the same as other but a lot of researchers use both as they tend to give slightly 
better performance.Both act as a regularizer, when they are used together slightly better regularization
happens as the noise/randomness introduced is from 2 different sources.



In [16]:
class BNDropoutReluNet(nn.Module):
    def __init__(self, P):
        super(BNDropoutReluNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv1_bn = nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.conv2_bn = nn.BatchNorm2d(50)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc1_bn = nn.BatchNorm1d(500)
        self.fc2 = nn.Linear(500, 10)
        self.dropout = nn.Dropout(p= P)
    def forward(self, x):
        x = self.conv1_bn(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = F.max_pool2d(x, 2, 2)
        x = self.conv2_bn(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = self.fc1_bn(F.relu(self.fc1(x)))
        x = self.dropout(x) 
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [17]:
model = BNDropoutReluNet(0.25).to(device)
optimizer = optim.SGD(model.parameters(), lr=00.01, momentum=0.5)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    print("Train set")
    test(model, device, train_loader)
    print("Test set")
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.388045
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.368524
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.927111
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.654746
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.472520
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.430616
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.409007
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.379685
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.364845
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.312162
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.231546
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.386107
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.246385
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.297396
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.213929
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.225092
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.211329
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.188647
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.128410
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.135397
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.006979
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.149816
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.121718
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.074483
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.062998
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.046983
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.060179
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.105064
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.154461
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.043692
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.054229
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.106295
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.078642
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.039674
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.089079
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.111245
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.040341
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.073468
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.114939
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.048367


Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.054367
Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.012681
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.046365
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.030897
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.036728
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.013215
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.092719
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.013109
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.097071
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.041188
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.032483
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.026150
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.022438
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.017731
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.038883
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.013068
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.033231
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.088457
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.102852
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.011958


In [18]:
class BNNet(nn.Module):
    def __init__(self):
        super(BNNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv1_bn = nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.conv2_bn = nn.BatchNorm2d(50)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc1_bn = nn.BatchNorm1d(500)
        self.fc2 = nn.Linear(500, 10)
    def forward(self, x):
        x = self.conv1_bn(F.relu(self.conv1(x)))
        x = F.max_pool2d(x, 2, 2)
        x = self.conv2_bn(F.relu(self.conv2(x)))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = self.fc1_bn(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [19]:
model = BNNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=00.01, momentum=0.5)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    print("Train set")
    test(model, device, train_loader)
    print("Test set")
    test(model, device, test_loader)   

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.406890
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.190988
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.605272
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.508731
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.398605
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.315375
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.414859
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.353143
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.245348
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.244984
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.248907
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.174629
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.283051
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.146647
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.185313
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.165262
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.160717
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.142445
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.188432
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.076800
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.091251
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.064731
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.122990
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.100904
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.030650
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.031708
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.059349
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.141048
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.025699
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.010360
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.030480
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.098820
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.164010
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.049069
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.025017
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.038874
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.008747
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.093434
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.025835
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.055935


Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.092803
Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.038355
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.053105
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.010622
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.013374
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.037784
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.020310
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.029252
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.033317
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.071486
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.022734
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.059631
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.022957
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.011330
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.042723
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.007180
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.006298
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.012088
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.011001
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.026645


part 4d)<br>
BatchNormalization does help in case of xavier, test error for non batch is 98% and for batch is 99%, batch 
normalization acts a regularizer which improves accuracy. Both and xavier and kaiming give nearly same test error kaiming gives really slightly better accuracy. Kaiming takes into account the activation function while xavier doesnt.


In [20]:
class BNXavierNet(nn.Module):
    def __init__(self):
        super(BNXavierNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        nn.init.xavier_normal_(self.conv1.weight)
        self.conv1_bn = nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        nn.init.xavier_normal_(self.conv2.weight)
        self.conv2_bn = nn.BatchNorm2d(50)
        self.fc1 = nn.Linear(4*4*50, 500)
        nn.init.xavier_normal_(self.fc1.weight)
        self.fc1_bn = nn.BatchNorm1d(500)
        self.fc2 = nn.Linear(500, 10)
        nn.init.xavier_normal_(self.fc2.weight)
    def forward(self, x):
        x = self.conv1_bn(F.relu(self.conv1(x)))
        x = F.max_pool2d(x, 2, 2)
        x = self.conv2_bn(F.relu(self.conv2(x)))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = self.fc1_bn(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [21]:
model = BNXavierNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=00.01, momentum=0.5)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    print("Train set")
    test(model, device, train_loader)
    print("Test set")
    test(model, device, test_loader)   

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302108
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.923637
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.783889
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.552521
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.271848
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.500243
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.232263
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.166350
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.311998
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.234075
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.281581
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.148035
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.187937
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.168796
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.133216
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.157053
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.099888
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.130404
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.163680
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.115377
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.069931
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.009849
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.040183
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.044818
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.075781
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.070617
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.017064
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.012912
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.032540
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.042525
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.026500
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.054756
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.056554
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.048801
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.089487
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.037164
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.071485
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.032637
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.022260
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.039087


Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.016313
Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.022445
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.011795
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.030526
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.011822
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.017172
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.016933
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.020341
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.010569
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.018207
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.023921
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.020352
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.075214
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.020562
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.018200
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.019267
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.017345
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.014279
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.022294
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.040602


In [22]:
class BNkaimingNet(nn.Module):
    def __init__(self):
        super(BNkaimingNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        nn.init.kaiming_normal_(self.conv1.weight)
        self.conv1_bn = nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        nn.init.kaiming_normal_(self.conv2.weight)
        self.conv2_bn = nn.BatchNorm2d(50)
        self.fc1 = nn.Linear(4*4*50, 500)
        nn.init.kaiming_normal_(self.fc1.weight)
        self.fc1_bn = nn.BatchNorm1d(500)
        self.fc2 = nn.Linear(500, 10)
        nn.init.kaiming_normal_(self.fc2.weight)
    def forward(self, x):
        x = self.conv1_bn(F.relu(self.conv1(x)))
        x = F.max_pool2d(x, 2, 2)
        x = self.conv2_bn(F.relu(self.conv2(x)))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = self.fc1_bn(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [23]:
model = BNkaimingNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=00.01, momentum=0.5)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    print("Train set")
    test(model, device, train_loader)
    print("Test set")
    test(model, device, test_loader)   

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.728354
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.349707
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.877292
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.716557
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.632806
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.480545
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.443849
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.468946
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.272577
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.553989
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.381731
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.377141
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.276520
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.326053
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.236252
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.198600
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.285331
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.233421
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.245279
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.143044
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.120302
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.060864
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.030763
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.136715
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.155291
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.109104
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.058074
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.048480
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.031393
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.050594
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.022178
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.066266
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.174048
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.062364
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.059743
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.094201
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.032503
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.050418
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.031693
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.132979


Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.030842
Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.083365
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.027090
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.025521
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.018934
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.026860
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.022355
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.071533
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.051616
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.019270
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.026085
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.199408
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.007039
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.035533
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.053807
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.012200
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.058834
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.056428
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.028588
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.061190


In [25]:
class XavierNet(nn.Module):
    def __init__(self):
        super(XavierNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        nn.init.xavier_normal_(self.conv1.weight)
#         self.conv1_bn = nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        nn.init.xavier_normal_(self.conv2.weight)
#         self.conv2_bn = nn.BatchNorm2d(50)
        self.fc1 = nn.Linear(4*4*50, 500)
        nn.init.xavier_normal_(self.fc1.weight)
#         self.fc1_bn = nn.BatchNorm1d(500)
        self.fc2 = nn.Linear(500, 10)
        nn.init.xavier_normal_(self.fc2.weight)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [26]:
model = XavierNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=00.01, momentum=0.5)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    print("Train set")
    test(model, device, train_loader)
    print("Test set")
    test(model, device, test_loader)   

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.324982
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.960906
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.251740
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.777318
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.530209
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.532334
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.636005
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.510438
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.437975
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.383703
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.224333
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.390755
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.286101
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.328297
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.403155
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.294999
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.486570
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.491603
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.270149
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.277704
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.187742
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.010009
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.026572
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.070577
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.136753
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.113327
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.021154
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.081440
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.094266
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.076674
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.111690
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.116335
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.056320
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.031650
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.090304
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.044729
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.034285
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.026604
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.090234
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.088317


Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.032566
Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.096490
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.027074
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.045238
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.011202
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.032490
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.086285
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.063158
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.033407
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.008721
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.019727
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.027017
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.027035
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.009885
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.007222
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.092908
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.010292
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.008157
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.026384
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.086400
